In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.

from pyspark.sql import SparkSession
from pyspark import SparkFiles

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Create a temporary view of the data frame
df.createOrReplaceTempView("home_sales")

spark.sql("SELECT * FROM home_sales").show()





+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

from pyspark.sql.functions import year, round, col, concat, lit, avg
import pyspark

# Convert the "date_built" column to a date type
df = df.withColumn('date_built', pyspark.sql.functions.to_date(concat(col('date_built'), lit('-01-01'))))

# Calculate the average price for a four-bedroom house sold in each year
result = df.filter(df['bedrooms'] == 4) \
           .withColumn('Year', year(df['date_built'])) \
           .groupBy('Year') \
           .agg(round(avg('price'), 2).alias('Average_Price')) \
           .orderBy('Year')

# Show the result
result.show()








+----+-------------+
|Year|Average_Price|
+----+-------------+
|2010|    296800.75|
|2011|     302141.9|
|2012|    298233.42|
|2013|    299999.39|
|2014|    299073.89|
|2015|    307908.86|
|2016|    296050.24|
|2017|    296576.69|
+----+-------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

from pyspark.sql.functions import year, round, avg

# Filter the DataFrame for homes with 3 bedrooms and 3 bathrooms
filtered_df = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3))

# Calculate the average price for each year the homes were built
result = filtered_df.groupBy(year('date_built').alias('Year')) \
                   .agg(round(avg('price'), 2).alias('Average_Price')) \
                   .orderBy('Year')

# Show the result
result.show()


+----+-------------+
|Year|Average_Price|
+----+-------------+
|2010|    292859.62|
|2011|    291117.47|
|2012|    293683.19|
|2013|    295962.27|
|2014|    290852.27|
|2015|     288770.3|
|2016|    290555.07|
|2017|    292676.79|
+----+-------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

from pyspark.sql.functions import year, round, avg

# Filter the DataFrame based on the specified criteria
filtered_df = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3) & (df['floors'] == 2) & (df['sqft_living'] >= 2000))

# Calculate the average price for each year the homes were built
result = filtered_df.groupBy(year('date_built').alias('Year')) \
                   .agg(round(avg('price'), 2).alias('Average_Price')) \
                   .orderBy('Year')

# Show the result
result.show()


+----+-------------+
|Year|Average_Price|
+----+-------------+
|2010|    285010.22|
|2011|    276553.81|
|2012|    307539.97|
|2013|    303676.79|
|2014|    298264.72|
|2015|    297609.97|
|2016|     293965.1|
|2017|    280317.58|
+----+-------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

from pyspark.sql.functions import round, avg

start_time = time.time()

filtered_df = df.filter(df['price'] >= 350000)

# Calculate the average price and "view" rating
result = filtered_df.agg(round(avg('price'), 2).alias('Average_Price'), avg('view').alias('Average_View'))

# Show the result
result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+------------+
|Average_Price|Average_View|
+-------------+------------+
|    473796.26|      32.264|
+-------------+------------+

--- 0.24137234687805176 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.

spark.catalog.cacheTable("home_sales")


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

# Filter the cached table for homes with a price greater than or equal to $350,000
filtered_table = spark.sql("SELECT * FROM home_sales_view WHERE price >= 350000")

# Calculate the average view rating
result = filtered_table.agg(avg('view').alias('Average_View'))

# Show the result
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+------------+
|Average_View|
+------------+
|      32.264|
+------------+

--- 0.38289546966552734 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 


output_path = "home_sales_partitioned"
df.write.mode("overwrite").partitionBy("date_built").parquet(output_path)





In [ ]:
# 11. Read the parquet formatted data.

partitioned_df = spark.read.parquet(output_path)

# Show the data in the partitioned DataFrame
partitioned_df.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|2015-01-01|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|2015-01-01|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|2015-01-01|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|2015-01-01|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|2015-01-01|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [ ]:
# 12. Create a temporary table for the parquet data.

partitioned_df.createOrReplaceTempView("parquet_table")

# Execute a SQL query on the temporary table
result = spark.sql("SELECT * FROM parquet_table")

# Show the data in the result DataFrame
result.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|2015-01-01|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|2015-01-01|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|2015-01-01|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|2015-01-01|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|2015-01-01|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

result = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM parquet_table WHERE price >= 350000 GROUP BY view")

# Show the data in the result DataFrame
result.show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.9821317195892334 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.


spark.catalog.uncacheTable("home_sales")


In [ ]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")
if is_cached:
    print("The home_sales table is still cached.")
else:
    print("The home_sales table is not cached.")


The home_sales table is not cached.
